In [ ]:
from flask import Flask, request, jsonify
import numpy as np
from keras.models import load_model

# Load LSTM model
model = load_model('lstm_model.h5')

# Set your desired threshold
threshold = 0.5

app = Flask(__name__)

@app.route('/')
def index():
    return "Pothole Detection"

@app.route('/predict', methods=['POST'])
def predict():
    try:
        content = request.get_json()

        # Validate incoming data
        required_fields = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'mag_x', 'mag_y', 'mag_z']
        for field in required_fields:
            if field not in content:
                raise ValueError(f"Missing required field: {field}")

        # Extract data
        accx, accy, accz, gyrox, gyroy, gyroz, magx, magy, magz = (
            content[field] for field in required_fields
        )

        # Assuming no scaling is needed
        input_query = np.array([[accx, accy, accz, gyrox, gyroy, gyroz, magx, magy, magz]])
        input_query = input_query.reshape(1, 9, 1)

        # Make predictions
        result = model.predict(input_query)[0][0]

        # Convert to binary prediction based on the threshold
        binary_prediction = 1 if result >= threshold else 0

        return jsonify({'ispothole': 'Yes' if binary_prediction == 1 else 'No'})

    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run(host='0.0.0.0')
